In [19]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk, scrolledtext
from PIL import ImageTk, Image
import time
import datetime, os
import matplotlib.pyplot as plt


In [20]:
def File_Choose():

    filename = filedialog.askopenfilename()
    filename = filename.replace('/', '\\')
    return filename

In [21]:
def Folder_Choose():
    folder_path = filedialog.askdirectory()
    folder_path = folder_path.replace('/', '\\')
    return(folder_path)

In [22]:
def start_point_get(event):
    global start_x, start_y # 声明写入全局变量
    Canvas_Image_Source.delete("rect1")  # 如果已经有“rect1”标记的图形，则删除
    # 在canvas 1上绘制一个矩形(rectangle是矩形的意思)
    Canvas_Image_Source.create_rectangle(event.x,
                             event.y,
                             event.x + 1,
                             event.y + 1,
                             outline="green",
                             width=5,
                             tag="rect1")
    # 在全局变量中存储坐标
    start_x, start_y = event.x, event.y

# 拖动中的事件- - - - - - - - - - - - - - - - - - - - - - - - - - -
def rect_drawing(event):
    # 拖动中的鼠标指针出现在区域外时的处理
    if event.x < 0:
        end_x = 0
    else:
        end_x = min(img_open.width, event.x)
    if event.y < 0:
        end_y = 0
    else:
        end_y = min(img_open.height, event.y)
    # 重新绘制“rect1”标签的图像
    Canvas_Image_Source.coords("rect1", start_x, start_y, end_x, end_y)

# 释放拖动时的事件- - - - - - - - - - - - - - - - - - - - - - - -
def release_action(event):
    global canvas1_size, canvas2_size, Image_Source
    canvas2_size = (400, 300)
    Img_Selected_Source =Image.open(Image_Source[0])

    # 将“rect1”标签的图像坐标恢复到原来的比例尺并获取
    start_x, start_y, end_x, end_y = [
        round(n) for n in Canvas_Image_Source.coords("rect1")
    ]
    ratio_canvas1 = img_ratio(Img_Selected_Source, canvas1_size)
    # 将所选区域恢复到实际图片尺寸
    Box = (start_x*ratio_canvas1, start_y*ratio_canvas1, end_x*ratio_canvas1, end_y*ratio_canvas1)


    Croped_Selected_Img = Img_Selected_Source.crop(Box) 
    ratio_canvas2 = img_ratio(Croped_Selected_Img, canvas2_size)
    Croped_Selected_Img_size_width, Croped_Selected_Img_size_height = Croped_Selected_Img.size
    Canvas2_Croped_Selected_Img = Croped_Selected_Img.resize((int(Croped_Selected_Img_size_width/ratio_canvas2), int(Croped_Selected_Img_size_height/ratio_canvas2)))
    print(ratio_canvas2, Box, (Croped_Selected_Img_size_width/ratio_canvas2, Croped_Selected_Img_size_height/ratio_canvas2))
    Opened_Canvas_Croped_Selected_Img = ImageTk.PhotoImage(Canvas2_Croped_Selected_Img)
    Canvas_Image_Selected_Region= tk.Canvas(Select_Pic_Region_Win, bg = 'gray')
    Canvas_Image_Selected_Region.place(x=canvas1_size[0]+2, y=0,width = canvas2_size[0], height = canvas2_size[1])
    Canvas_Image_Selected_Region.create_image(0, 0, image=Opened_Canvas_Croped_Selected_Img, anchor=tk.NW)
    Canvas_Image_Selected_Region.image = Opened_Canvas_Croped_Selected_Img

In [23]:
def Select_Pic_Region(canvas):
    canvas.bind("<ButtonPress-1>", start_point_get)
    canvas.bind("<Button1-Motion>", rect_drawing)
    canvas.bind("<ButtonRelease-1>", release_action)

In [24]:
def img_ratio(img, canvas_size):
    w, h = img.size
    w_ratio = w/canvas_size[0]
    h_ratio = h/canvas_size[1]
    if w_ratio > h_ratio:
        ratio = w_ratio
    else:
        ratio = h_ratio
    return ratio

In [25]:
def choosepic(image_label):
    global img_open, canvas1_size, Image_Source
    Image_Source = ['']
    path_ = filedialog.askopenfilename()
    Image_Source[0] = path_
    img_open = Image.open(path_)
    w, h = img_open.size
    ratio_canvas1 = img_ratio(img_open, canvas1_size)
    img_open = img_open.resize((int(w/ratio_canvas1), int(h/ratio_canvas1)))
    img = ImageTk.PhotoImage(img_open)
    image_label.create_image(0, 0, image=img, anchor=tk.NW)
    image_label.image = img

In [26]:
global canvas1_size 
canvas1_size = (800, 500)
Select_Pic_Region_Win = tk.Toplevel()
Select_Pic_Region_Win.title('选择图片识别区域')
Select_Pic_Region_Win.geometry('1902x1080')
Select_Pic_Region_Win.config(bg='aliceblue')
Select_Pic_Region_Win.focus()
tk.Button(Select_Pic_Region_Win, text = '打开图片', font='黑体 28 bold', command=lambda: choosepic(Canvas_Image_Source)).place(x = 100, y = 830,w = 250, h = 80)
tk.Button(Select_Pic_Region_Win, text = '选择区域', font='黑体 28 bold', command=lambda: Select_Pic_Region(Canvas_Image_Source)).place(x = 400, y = 830,w = 250, h = 80)
tk.Button(Select_Pic_Region_Win, text = '确认', font='黑体 28 bold', command=lambda: Select_Pic_Region(Canvas_Image_Source)).place(x = 700, y = 830,w = 250, h = 80)
Canvas_Image_Source = tk.Canvas(Select_Pic_Region_Win, bg = 'gray')
Canvas_Image_Source.place(x=0, y=0,width = canvas1_size[0], height = canvas1_size[1])
Select_Pic_Region_Win.update()

In [27]:
Select_Pic_Region_Win.update()
Select_Pic_Region_Win.mainloop()

1.59 (290.4, 249.6, 926.4, 715.1999999999999) (400.0, 292.45283018867923)
2.84 (283.2, 187.2, 1344.0, 1039.2) (373.59154929577466, 300.0)
1.8333333333333333 (300.0, 300.0, 967.1999999999999, 849.6) (363.8181818181818, 300.0)
2.6633333333333336 (316.8, 144.0, 1072.8, 943.1999999999999) (283.85481852315394, 300.0)
2.0225 (252.0, 285.59999999999997, 1060.8, 864.0) (400.0, 285.7849196538937)
2.6966666666666668 (252.0, 134.4, 1027.2, 943.1999999999999) (287.39184177997527, 300.0)
2.5533333333333332 (348.0, 194.4, 969.5999999999999, 960.0) (243.60313315926894, 300.0)
2.32 (300.0, 271.2, 1147.2, 967.1999999999999) (365.08620689655174, 300.0)
2.4 (302.4, 232.79999999999998, 993.5999999999999, 952.8) (288.33333333333337, 300.0)
2.033333333333333 (348.0, 218.4, 967.1999999999999, 828.0) (304.42622950819674, 300.0)
2.0233333333333334 (170.4, 129.6, 871.1999999999999, 736.8) (346.45799011532125, 300.0)
2.4566666666666666 (283.2, 139.2, 976.8, 876.0) (282.4966078697422, 300.0)
2.5366666666666666 (3